In [6]:
import projectpath
import pandas as pd 
import d01_sql as database_calls
#import cufflinks as cf 
import matplotlib.pyplot as plt 
import seaborn as sns 
from matplotlib.patches import Polygon
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Image, display, clear_output
import warnings
import ipywidgets as wg 
from tkinter import *
from tkinter import ttk, messagebox
from IPython.display import display
InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings("ignore")
%matplotlib inline 

In [7]:
#READ FILES 
e2e_path = '../data/data_e2e/'
read_db_path = 'db/'

#db-path 
dbpath = e2e_path+read_db_path+'sqlp1.db'

#establish connection again
conn = database_calls.create_database(dbpath)
#establush cursor 
cur = conn.cursor()

#tables 
tables = ["company","financialinformation","security_price"]

In [8]:
#show tables function 
def showtable(cur,tables):
    cur.execute("SELECT * FROM "+tables)
    rows = cur.fetchall()
    names = list(map(lambda x: x[0], cur.description))
    show_table = pd.DataFrame(columns=names)
    show_table.rows = rows 
    for i in range(len(rows)): 
        show_table.loc[i] = rows[i]
    return show_table

In [9]:
securities_table = showtable(cur,"security_price")
securities_table 

KeyboardInterrupt: 

In [ ]:
company_table = showtable(cur,"company")
company_table 

,id,ticker,name,market,city
0,0,AAPL,Apple Inc.,us_market,Cupertino
1,1,MSFT,Microsoft Corporation,us_market,Redmond
2,2,AMZN,"Amazon.com, Inc.",us_market,Seattle
3,3,GOOG,Alphabet Inc.,us_market,Mountain View
4,4,GOOGL,Alphabet Inc.,us_market,Mountain View


In [ ]:
financialinformation_table = showtable(cur,"financialinformation")
financialinformation_table 

,id,currency,market_cap,average_volume_traded,fifty_two_wk_high,fifty_two_wk_low,current_price,fifty_two_wk_change,divident_yield
0,0,b'USD',2572687507456,72367719,165.70,116.21,156.81,0.317178,0.0056
1,1,b'USD',2475230756864,24217242,349.67,209.11,329.68,0.540057,0.0075
2,2,b'USD',1777330683904,2991297,3773.08,2881.00,3504.56,0.106223,0.0000
3,3,b'USD',1891476963328,849606,3037.00,1699.00,2856.12,0.622113,0.0000
4,4,b'USD',1891477356544,1523462,3019.33,1694.00,2843.66,0.620873,0.0000


In [11]:
import sqlite3 as sql
# connecting sqlite database
con = sql.connect(dbpath)
# if success
if con:
    print("Database connected")
# creating cursor for performing queries on database
cur = con.cursor()
cur.execute('SELECT Distinct Date FROM security_price;')
# getting the data in d variable
d=cur.fetchall()
# list for stroing dates
date=[]

# appending all dates in date list
for i in range(len(d)):
    date.append(d[i][0])

# creating an Tkinter window
window = Tk()
# setting window title
window.title("Search Data")
# setting width x height + x pos + y pos
window.geometry("1300x580+0+0")
# defining background color
color = 'lightblue'
# setting bg color
window.config(bg=color)

##############
# Table Grid #
#############
# Styling the headings of table  
style = ttk.Style()
style.configure("mystyle.Treeview", highlightthickness=5, bd=5, font=('Arial', 10))  # Modify the font of the body
style.configure("mystyle.Treeview.Heading", font=('Calibri', 13, 'bold'))  # Modify the font of the headings
##############################
# creating frame for table
Table_Section = LabelFrame(window, text='', pady=40, padx=25,
                           font=('Arial', '20', 'bold'), bg=color)
Table_Section.pack(side=TOP, anchor=CENTER, pady=20)
# creating frame inside the table frame for inputs and buttons
Search_frame = Frame(Table_Section, bg=color)
Search_frame.pack(side=TOP, anchor=NW)
# Ticket text Label 
Label(Search_frame, text='Ticket', font=('Arial', '12', 'bold'), bg=color).grid(row=0, column=0, sticky=W, padx=3)
# Ticket dropdown list
ticket_list = ttk.Combobox(Search_frame, values=('MSFT', 'AAPL', 'AMZN', 'GOOG'), font=('Arial', '16', 'bold'))
ticket_list.grid(row=0, column=1, padx=3, pady=5)
ticket_list.current(0)  # setting it to disply the first value in ticket dropdown
# button for search by ticket
Button(Search_frame, text='Search By Ticket', font=('Arial', '12', 'bold'),
       command=lambda: get_ticket(ticket_list.get()),
       width=15, height=1).grid(row=0, column=2, padx=3, columnspan=2)
# From Date text Label 
Label(Search_frame, text='From Date', font=('Arial', '12', 'bold'), bg=color).grid(row=1, column=0, sticky=W, padx=3)
# from date dropdown list
from_Date_list = ttk.Combobox(Search_frame, values=date, font=('Arial', '16', 'bold'))
from_Date_list.grid(row=1, column=1, padx=3, pady=5)
from_Date_list.current(0)  # setting it to disply the first value in from date dropdown
# To Date text Label 
Label(Search_frame, text='To Date', font=('Arial', '12', 'bold'), bg=color).grid(row=1, column=2, sticky=W, padx=3)
# to date dropdown list
to_Date_list = ttk.Combobox(Search_frame, values=date, font=('Arial', '16', 'bold'))
to_Date_list.grid(row=1, column=3, padx=3, pady=5)
to_Date_list.current(0)  # setting it to disply the first value in to date dropdown
# button for search by Date
Button(Search_frame, text='Search By Date', font=('Arial', '12', 'bold'),
       command=lambda: get_ticket_date(ticket_list.get(), from_Date_list.get(), to_Date_list.get() ),
       width=12, height=1).grid(row=1, column=4, padx=3)
# frame for table
Table_Frame = Frame(Table_Section, bg=color, pady=3, bd=3)
# adding scrollbar at Y axis
scrollbar = Scrollbar(Table_Frame, orient=VERTICAL)
# Creating a table or data viewer or tree view
Data_tree = ttk.Treeview(Table_Frame, columns=(1, 2, 3, 4, 5, 6, 7, 8), show='headings',
                         height=18, yscrollcommand=scrollbar.set, style="mystyle.Treeview")
# setting column width in Treeview
Data_tree.column(1, width=150, minwidth=50, stretch=NO, anchor=CENTER)
Data_tree.column(2, width=150, minwidth=100, stretch=NO, anchor=CENTER)
Data_tree.column(3, width=150, minwidth=100, stretch=NO, anchor=CENTER)
Data_tree.column(4, width=150, minwidth=100, stretch=NO, anchor=CENTER)
Data_tree.column(5, width=150, minwidth=100, stretch=NO, anchor=CENTER)
Data_tree.column(6, width=150, minwidth=100, stretch=NO, anchor=CENTER)
Data_tree.column(7, width=150, minwidth=100, stretch=NO, anchor=CENTER)
Data_tree.column(8, width=150, minwidth=100, stretch=NO, anchor=CENTER)
# setting the headings 
Data_tree.heading(1, text="Ticket")
Data_tree.heading(2, text="Date")
Data_tree.heading(3, text="Open")
Data_tree.heading(4, text="High")
Data_tree.heading(5, text="Low")
Data_tree.heading(6, text="close")
Data_tree.heading(7, text="Volume")
Data_tree.heading(8, text="Adj_close")
# configure the scrollbar at Y axis
scrollbar.config(command=Data_tree.yview)
scrollbar.pack(side=RIGHT, fill=Y)
Data_tree.pack()
Table_Frame.pack(side=TOP, anchor=CENTER)

# function for query by ticket
def get_ticket(Ticket_name):

    cur.execute("select ticker, date, open, high, low, close, volume, adj_close from security_price, company "
                "where company.id = security_price.id and ticker =?", (Ticket_name,))
    res = cur.fetchall()
    # when data found
    if res:
        # clearing previous data from tree viewer
        clear_table()
        for row in res:
            # inserting data in table viwer
            Data_tree.insert('', END, values=row)
    else:
        # showing msg if no data found
        messagebox.showwarning("Alert:", "Data Not Found...")

        
def get_ticket_date(Ticket_name, from_date, to_date):
    cur.execute("select ticker, date, open, high, low, close, volume, adj_close from security_price, company "
                "where company.id = security_price.id and ticker =? and (Date between ? and ? );", (Ticket_name,from_date, to_date))
    res = cur.fetchall()
    # when data found
    if res:
        # clearing previous data from tree viewer
        clear_table()
        for row in res:
             # inserting data in table viwer
            Data_tree.insert('', END, values=row)
    else:
        # showing msg if no data found
        messagebox.showwarning("Alert:", "Data Not Found...")

# function for clearing data tree viwer
def clear_table():
    for i in Data_tree.get_children():
        Data_tree.delete(i)


window.mainloop()


Database connected


''

''

''

''

...

{}

{}

{}

{}

{}

In [ ]:

'''
Frm = Frame(ws)

Label(Frm,text='Enter Search Here:').pack(side=LEFT)

modify = Entry(Frm)

modify.pack(side=LEFT, fill=BOTH, expand=1)

modify.focus_set()

buttn = Button(Frm, text='Find')
buttn.pack(side=RIGHT)
Frm.pack(side=TOP)

txt = Text(ws)

txt.insert('1.0','Enter here...')
txt.pack(side=BOTTOM)


def find():
	
	txt.tag_remove('found', '1.0', END)
	ser = modify.get()
	if ser:
		idx = '1.0'
		while 1:
			idx = txt.search(ser, idx, nocase=1,
							stopindex=END)
			if not idx: break
			lastidx = '%s+%dc' % (idx, len(ser))
			
			txt.tag_add('found', idx, lastidx)
			idx = lastidx
		txt.tag_config('found', foreground='blue')
	modify.focus_set()
buttn.config(command=find)
'''

"\nFrm = Frame(ws)\n\nLabel(Frm,text='Enter Search Here:').pack(side=LEFT)\n\nmodify = Entry(Frm)\n\nmodify.pack(side=LEFT, fill=BOTH, expand=1)\n\nmodify.focus_set()\n\nbuttn = Button(Frm, text='Find')\nbuttn.pack(side=RIGHT)\nFrm.pack(side=TOP)\n\ntxt = Text(ws)\n\ntxt.insert('1.0','Enter here...')\ntxt.pack(side=BOTTOM)\n\n\ndef find():\n\t\n\ttxt.tag_remove('found', '1.0', END)\n\tser = modify.get()\n\tif ser:\n\t\tidx = '1.0'\n\t\twhile 1:\n\t\t\tidx = txt.search(ser, idx, nocase=1,\n\t\t\t\t\t\t\tstopindex=END)\n\t\t\tif not idx: break\n\t\t\tlastidx = '%s+%dc' % (idx, len(ser))\n\t\t\t\n\t\t\ttxt.tag_add('found', idx, lastidx)\n\t\t\tidx = lastidx\n\t\ttxt.tag_config('found', foreground='blue')\n\tmodify.focus_set()\nbuttn.config(command=find)\n"